In [1]:
!pip install pyspark
# Alternatively, if you want to install a specific version of pyspark:
#!pip install pyspark==3.2.1 

In [2]:
from tqdm import tqdm

import requests
import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as f
from pyspark.sql.functions import split, regexp_replace, year, month, dayofmonth, to_timestamp

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator #, BinaryClassificationEvaluator 

# Basic libreries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Pre-processing phase
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Features Importance
from sklearn.inspection import permutation_importance

# Model
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# Hyper-Parameter Tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score

# Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
# Create the session
conf = SparkConf().set("spark.ui.port", "4050").set('spark.executor.memory', '4G').set('spark.driver.memory', '45G').set('spark.driver.maxResultSize', '10G')

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [4]:
sc._conf.getAll()

[('spark.app.id', 'local-1652722929906'),
 ('spark.driver.memory', '45G'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.maxResultSize', '10G'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.app.startTime', '1652722927634'),
 ('spark.ui.port', '4050'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'e77e74af1ccf'),
 ('spark.executor.memory', '4G'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '44825')]

In [5]:
spark

In [6]:
GDRIVE_DIR = "/content/gdrive" # Your own mount point on Google Drive
GDRIVE_HOME_DIR = GDRIVE_DIR + "/My Drive" # Your own home directory
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR +  "/Sapienza/Primo Anno/Big Data Computing/Project"

# Point Colaboratory to our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

DATASET_URL = "https://raw.githubusercontent.com/AndreaBe99/big-data-project/main/data/dataframe.csv"
GDRIVE_DATASET_FILE = GDRIVE_DATA_DIR + "/" + DATASET_URL.split("/")[-1]

import requests

"""
This function downloads a file from a specific URL directly to Google Drive.
"""
def get_data(dataset_url, dest, chunk_size=1024):
  response = requests.get(dataset_url, stream=True)
  if response.status_code == 200: # Test if everything went ok
    with open(dest, "wb") as file:
      for block in response.iter_content(chunk_size=chunk_size): 
        if block: 
          file.write(block)

print("Retrieving dataset from URL: {} ...".format(DATASET_URL))
get_data(DATASET_URL, GDRIVE_DATASET_FILE)
print("Dataset successfully retrieved and stored at: {}".format(GDRIVE_DATASET_FILE))

Mounted at /content/gdrive
Retrieving dataset from URL: https://raw.githubusercontent.com/AndreaBe99/big-data-project/main/data/dataframe.csv ...
Dataset successfully retrieved and stored at: /content/gdrive/My Drive/Sapienza/Primo Anno/Big Data Computing/Project/dataframe.csv


In [7]:
spotify_tracks = spark.read.load(GDRIVE_DATASET_FILE, 
                           format="csv", 
                           sep=";", 
                           inferSchema="true", 
                           header="true")

In [8]:
spotify_tracks.cache()

DataFrame[id: string, track_name: string, track_explicit: boolean, track_popularity: int, album_name: string, album_release_date: string, album_release_date_precision: string, artist_name: string, audio_avg_pitches: string, audio_avg_timbre: string, audio_acousticness: double, audio_danceability: double, audio_duration_ms: int, audio_energy: double, audio_instrumentalness: double, audio_key_1: int, audio_liveness: double, audio_loudness: double, audio_mode_1: int, audio_speechiness: double, audio_tempo: double, audio_time_signature: int, audio_valence: double, track_uri: string, track_genre: string]

### **Check the shape of the loaded dataset, i.e., number of rows and columns**

In [9]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(spotify_tracks.count(), len(spotify_tracks.columns)))

The shape of the dataset is 79339 rows by 25 columns


### **Print out the schema of the loaded dataset**

In [10]:
spotify_tracks.printSchema()

root
 |-- id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_explicit: boolean (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- album_name: string (nullable = true)
 |-- album_release_date: string (nullable = true)
 |-- album_release_date_precision: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- audio_avg_pitches: string (nullable = true)
 |-- audio_avg_timbre: string (nullable = true)
 |-- audio_acousticness: double (nullable = true)
 |-- audio_danceability: double (nullable = true)
 |-- audio_duration_ms: integer (nullable = true)
 |-- audio_energy: double (nullable = true)
 |-- audio_instrumentalness: double (nullable = true)
 |-- audio_key_1: integer (nullable = true)
 |-- audio_liveness: double (nullable = true)
 |-- audio_loudness: double (nullable = true)
 |-- audio_mode_1: integer (nullable = true)
 |-- audio_speechiness: double (nullable = true)
 |-- audio_tempo: double (nullable = true)
 |-- audio

In [11]:
spotify_tracks.show(5)

+--------------------+-----------------+--------------+----------------+--------------------+------------------+----------------------------+---------------+--------------------+--------------------+------------------+------------------+-----------------+------------+----------------------+-----------+--------------+--------------+------------+-----------------+-----------+--------------------+-------------+--------------------+-----------+
|                  id|       track_name|track_explicit|track_popularity|          album_name|album_release_date|album_release_date_precision|    artist_name|   audio_avg_pitches|    audio_avg_timbre|audio_acousticness|audio_danceability|audio_duration_ms|audio_energy|audio_instrumentalness|audio_key_1|audio_liveness|audio_loudness|audio_mode_1|audio_speechiness|audio_tempo|audio_time_signature|audio_valence|           track_uri|track_genre|
+--------------------+-----------------+--------------+----------------+--------------------+-----------------

In [12]:
spotify_tracks = spotify_tracks.dropDuplicates()

In [13]:
# Split String audio_avg_pitches to n columns
spotify_tracks = spotify_tracks.select(spotify_tracks.columns + [f.translate(f.col("audio_avg_pitches"), "[]", "").alias("audio_avg_pitches_list")])
spotify_tracks = spotify_tracks.withColumn("pitch_1", split(col("audio_avg_pitches_list"), ", ").getItem(0))\
                                .withColumn("pitch_2", split(col("audio_avg_pitches_list"), ", ").getItem(1))\
                                .withColumn("pitch_3", split(col("audio_avg_pitches_list"), ", ").getItem(2))\
                                .withColumn("pitch_4", split(col("audio_avg_pitches_list"), ", ").getItem(3))\
                                .withColumn("pitch_5", split(col("audio_avg_pitches_list"), ", ").getItem(4))\
                                .withColumn("pitch_6", split(col("audio_avg_pitches_list"), ", ").getItem(5))\
                                .withColumn("pitch_7", split(col("audio_avg_pitches_list"), ", ").getItem(6))\
                                .withColumn("pitch_8", split(col("audio_avg_pitches_list"), ", ").getItem(7))\
                                .withColumn("pitch_9", split(col("audio_avg_pitches_list"), ", ").getItem(8))\
                                .withColumn("pitch_10", split(col("audio_avg_pitches_list"), ", ").getItem(9))\
                                .withColumn("pitch_11", split(col("audio_avg_pitches_list"), ", ").getItem(10))\
                                .withColumn("pitch_12", split(col("audio_avg_pitches_list"), ", ").getItem(11))
spotify_tracks.printSchema()

root
 |-- id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_explicit: boolean (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- album_name: string (nullable = true)
 |-- album_release_date: string (nullable = true)
 |-- album_release_date_precision: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- audio_avg_pitches: string (nullable = true)
 |-- audio_avg_timbre: string (nullable = true)
 |-- audio_acousticness: double (nullable = true)
 |-- audio_danceability: double (nullable = true)
 |-- audio_duration_ms: integer (nullable = true)
 |-- audio_energy: double (nullable = true)
 |-- audio_instrumentalness: double (nullable = true)
 |-- audio_key_1: integer (nullable = true)
 |-- audio_liveness: double (nullable = true)
 |-- audio_loudness: double (nullable = true)
 |-- audio_mode_1: integer (nullable = true)
 |-- audio_speechiness: double (nullable = true)
 |-- audio_tempo: double (nullable = true)
 |-- audio

In [14]:
spotify_tracks = spotify_tracks.dropDuplicates()

In [15]:
# Split String audio_avg_timbre to n columns
spotify_tracks = spotify_tracks.select(spotify_tracks.columns + [f.translate(f.col("audio_avg_timbre"), "[]", "").alias("audio_avg_timbre_list")])
spotify_tracks = spotify_tracks.withColumn("timbre_1", split(col("audio_avg_timbre_list"), ", ").getItem(0))\
                                .withColumn("timbre_2", split(col("audio_avg_timbre_list"), ", ").getItem(1))\
                                .withColumn("timbre_3", split(col("audio_avg_timbre_list"), ", ").getItem(2))\
                                .withColumn("timbre_4", split(col("audio_avg_timbre_list"), ", ").getItem(3))\
                                .withColumn("timbre_5", split(col("audio_avg_timbre_list"), ", ").getItem(4))\
                                .withColumn("timbre_6", split(col("audio_avg_timbre_list"), ", ").getItem(5))\
                                .withColumn("timbre_7", split(col("audio_avg_timbre_list"), ", ").getItem(6))\
                                .withColumn("timbre_8", split(col("audio_avg_timbre_list"), ", ").getItem(7))\
                                .withColumn("timbre_9", split(col("audio_avg_timbre_list"), ", ").getItem(8))\
                                .withColumn("timbre_10", split(col("audio_avg_timbre_list"), ", ").getItem(9))\
                                .withColumn("timbre_11", split(col("audio_avg_timbre_list"), ", ").getItem(10))\
                                .withColumn("timbre_12", split(col("audio_avg_timbre_list"), ", ").getItem(11))
spotify_tracks.printSchema()

root
 |-- id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_explicit: boolean (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- album_name: string (nullable = true)
 |-- album_release_date: string (nullable = true)
 |-- album_release_date_precision: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- audio_avg_pitches: string (nullable = true)
 |-- audio_avg_timbre: string (nullable = true)
 |-- audio_acousticness: double (nullable = true)
 |-- audio_danceability: double (nullable = true)
 |-- audio_duration_ms: integer (nullable = true)
 |-- audio_energy: double (nullable = true)
 |-- audio_instrumentalness: double (nullable = true)
 |-- audio_key_1: integer (nullable = true)
 |-- audio_liveness: double (nullable = true)
 |-- audio_loudness: double (nullable = true)
 |-- audio_mode_1: integer (nullable = true)
 |-- audio_speechiness: double (nullable = true)
 |-- audio_tempo: double (nullable = true)
 |-- audio

In [16]:
# String to Date but return null value 
# spotify_tracks_dt = spotify_tracks.withColumn("album_release_date", to_date(col("album_release_date"),"yyyy-MM-dd"))
# spotify_tracks_dt.select("album_release_date").show()

# Convert String to date
def to_date_(col, formats=("yyyy-MM-dd", "y")):
    # Spark 2.2 or later syntax, for < 2.2 use unix_timestamp and cast
    return coalesce(*[to_date(col, f) for f in formats])

spotify_tracks = spotify_tracks.withColumn("album_release_date_td", to_date_("album_release_date"))
spotify_tracks.select("album_release_date_td").show()

+---------------------+
|album_release_date_td|
+---------------------+
|           2014-05-30|
|           2011-01-01|
|           2003-02-18|
|           2011-02-28|
|           2013-01-01|
|           2013-04-30|
|           2014-02-18|
|           2007-03-20|
|           2010-01-01|
|           2011-01-01|
|           2012-01-24|
|           1998-10-01|
|           2012-09-24|
|           2013-08-28|
|           2006-08-31|
|           2013-01-21|
|           2008-04-18|
|           2013-02-13|
|           2016-09-14|
|           2012-11-21|
+---------------------+
only showing top 20 rows



In [17]:
# Split date to columns
spotify_tracks = spotify_tracks.withColumn('day', dayofmonth(col('album_release_date_td')))
spotify_tracks = spotify_tracks.withColumn('month', month(col('album_release_date_td')))
spotify_tracks = spotify_tracks.withColumn('year', year(col('album_release_date_td')))

In [18]:
spotify_tracks.printSchema()

root
 |-- id: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- track_explicit: boolean (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- album_name: string (nullable = true)
 |-- album_release_date: string (nullable = true)
 |-- album_release_date_precision: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- audio_avg_pitches: string (nullable = true)
 |-- audio_avg_timbre: string (nullable = true)
 |-- audio_acousticness: double (nullable = true)
 |-- audio_danceability: double (nullable = true)
 |-- audio_duration_ms: integer (nullable = true)
 |-- audio_energy: double (nullable = true)
 |-- audio_instrumentalness: double (nullable = true)
 |-- audio_key_1: integer (nullable = true)
 |-- audio_liveness: double (nullable = true)
 |-- audio_loudness: double (nullable = true)
 |-- audio_mode_1: integer (nullable = true)
 |-- audio_speechiness: double (nullable = true)
 |-- audio_tempo: double (nullable = true)
 |-- audio

In [19]:
# Delete columns
cols = ("album_release_date", "album_release_date_td", "audio_avg_pitches","audio_avg_timbre", "audio_avg_pitches_list", "audio_avg_timbre_list", "track_uri", "id")
spotify_tracks = spotify_tracks.drop(*cols)

spotify_tracks.printSchema()

root
 |-- track_name: string (nullable = true)
 |-- track_explicit: boolean (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- album_name: string (nullable = true)
 |-- album_release_date_precision: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- audio_acousticness: double (nullable = true)
 |-- audio_danceability: double (nullable = true)
 |-- audio_duration_ms: integer (nullable = true)
 |-- audio_energy: double (nullable = true)
 |-- audio_instrumentalness: double (nullable = true)
 |-- audio_key_1: integer (nullable = true)
 |-- audio_liveness: double (nullable = true)
 |-- audio_loudness: double (nullable = true)
 |-- audio_mode_1: integer (nullable = true)
 |-- audio_speechiness: double (nullable = true)
 |-- audio_tempo: double (nullable = true)
 |-- audio_time_signature: integer (nullable = true)
 |-- audio_valence: double (nullable = true)
 |-- track_genre: string (nullable = true)
 |-- pitch_1: string (nullable = true)
 |-- pitch_

In [20]:
# Cast converted columns
spotify_tracks = spotify_tracks.withColumn("pitch_1", spotify_tracks.pitch_1.cast('double'))\
                                .withColumn("pitch_2", spotify_tracks.pitch_2.cast('double'))\
                                .withColumn("pitch_3", spotify_tracks.pitch_3.cast('double'))\
                                .withColumn("pitch_4", spotify_tracks.pitch_4.cast('double'))\
                                .withColumn("pitch_5", spotify_tracks.pitch_5.cast('double'))\
                                .withColumn("pitch_6", spotify_tracks.pitch_6.cast('double'))\
                                .withColumn("pitch_7", spotify_tracks.pitch_7.cast('double'))\
                                .withColumn("pitch_8", spotify_tracks.pitch_8.cast('double'))\
                                .withColumn("pitch_9", spotify_tracks.pitch_9.cast('double'))\
                                .withColumn("pitch_10", spotify_tracks.pitch_10.cast('double'))\
                                .withColumn("pitch_11", spotify_tracks.pitch_11.cast('double'))\
                                .withColumn("pitch_12", spotify_tracks.pitch_12.cast('double'))

spotify_tracks = spotify_tracks.withColumn("timbre_1", spotify_tracks.timbre_1.cast('double'))\
                                .withColumn("timbre_2", spotify_tracks.timbre_2.cast('double'))\
                                .withColumn("timbre_3", spotify_tracks.timbre_3.cast('double'))\
                                .withColumn("timbre_4", spotify_tracks.timbre_4.cast('double'))\
                                .withColumn("timbre_5", spotify_tracks.timbre_5.cast('double'))\
                                .withColumn("timbre_6", spotify_tracks.timbre_6.cast('double'))\
                                .withColumn("timbre_7", spotify_tracks.timbre_7.cast('double'))\
                                .withColumn("timbre_8", spotify_tracks.timbre_8.cast('double'))\
                                .withColumn("timbre_9", spotify_tracks.timbre_9.cast('double'))\
                                .withColumn("timbre_10", spotify_tracks.timbre_10.cast('double'))\
                                .withColumn("timbre_11", spotify_tracks.timbre_11.cast('double'))\
                                .withColumn("timbre_12", spotify_tracks.timbre_12.cast('double'))

spotify_tracks = spotify_tracks.withColumn("year", spotify_tracks.year.cast('int'))\
                                .withColumn("day", spotify_tracks.day.cast('int'))\
                                .withColumn("month", spotify_tracks.month.cast('int'))

spotify_tracks.printSchema()

root
 |-- track_name: string (nullable = true)
 |-- track_explicit: boolean (nullable = true)
 |-- track_popularity: integer (nullable = true)
 |-- album_name: string (nullable = true)
 |-- album_release_date_precision: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- audio_acousticness: double (nullable = true)
 |-- audio_danceability: double (nullable = true)
 |-- audio_duration_ms: integer (nullable = true)
 |-- audio_energy: double (nullable = true)
 |-- audio_instrumentalness: double (nullable = true)
 |-- audio_key_1: integer (nullable = true)
 |-- audio_liveness: double (nullable = true)
 |-- audio_loudness: double (nullable = true)
 |-- audio_mode_1: integer (nullable = true)
 |-- audio_speechiness: double (nullable = true)
 |-- audio_tempo: double (nullable = true)
 |-- audio_time_signature: integer (nullable = true)
 |-- audio_valence: double (nullable = true)
 |-- track_genre: string (nullable = true)
 |-- pitch_1: double (nullable = true)
 |-- pitch_

In [21]:
#temp = spotify_tracks.groupby("track_genre").count()
#new_train = spotify_tracks.join(temp, "track_genre", how = 'leftouter')
#num_labels = spotify_tracks.select(countDistinct(spotify_tracks.score)).first()[0]
#spotify_tracks = new_train.withColumn("weight",(new_train.count()/(num_labels * new_train["count"])))
    
#spotify_tracks.show(truncate=False)

In [22]:
# Let's define some constants which we will use throughout this notebook
NUMERICAL_FEATURES = []
CATEGORICAL_FEATURES = []
TARGET_VARIABLE = "track_genre"

#Get All column names and it's types
for col in spotify_tracks.dtypes:
    if col[1] == "string":
        CATEGORICAL_FEATURES.append(col[0])
    else:
        NUMERICAL_FEATURES.append(col[0])

CATEGORICAL_FEATURES.remove(TARGET_VARIABLE)
print("Categorical: ", CATEGORICAL_FEATURES, "\nNumerical: ", NUMERICAL_FEATURES)

Categorical:  ['track_name', 'album_name', 'album_release_date_precision', 'artist_name'] 
Numerical:  ['track_explicit', 'track_popularity', 'audio_acousticness', 'audio_danceability', 'audio_duration_ms', 'audio_energy', 'audio_instrumentalness', 'audio_key_1', 'audio_liveness', 'audio_loudness', 'audio_mode_1', 'audio_speechiness', 'audio_tempo', 'audio_time_signature', 'audio_valence', 'pitch_1', 'pitch_2', 'pitch_3', 'pitch_4', 'pitch_5', 'pitch_6', 'pitch_7', 'pitch_8', 'pitch_9', 'pitch_10', 'pitch_11', 'pitch_12', 'timbre_1', 'timbre_2', 'timbre_3', 'timbre_4', 'timbre_5', 'timbre_6', 'timbre_7', 'timbre_8', 'timbre_9', 'timbre_10', 'timbre_11', 'timbre_12', 'day', 'month', 'year']


In [23]:
spotify_tracks.cache()

DataFrame[track_name: string, track_explicit: boolean, track_popularity: int, album_name: string, album_release_date_precision: string, artist_name: string, audio_acousticness: double, audio_danceability: double, audio_duration_ms: int, audio_energy: double, audio_instrumentalness: double, audio_key_1: int, audio_liveness: double, audio_loudness: double, audio_mode_1: int, audio_speechiness: double, audio_tempo: double, audio_time_signature: int, audio_valence: double, track_genre: string, pitch_1: double, pitch_2: double, pitch_3: double, pitch_4: double, pitch_5: double, pitch_6: double, pitch_7: double, pitch_8: double, pitch_9: double, pitch_10: double, pitch_11: double, pitch_12: double, timbre_1: double, timbre_2: double, timbre_3: double, timbre_4: double, timbre_5: double, timbre_6: double, timbre_7: double, timbre_8: double, timbre_9: double, timbre_10: double, timbre_11: double, timbre_12: double, day: int, month: int, year: int]

In [24]:
spotify_tracks.rdd.getNumPartitions()

200

In [25]:
#spotify_tracks = spotify_tracks.repartition(500)
##spotify_tracks = spotify_tracks_rp.coalesce(1)
#spotify_tracks.cache()
#spotify_tracks.rdd.getNumPartitions()

In [26]:
# null values in each column
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
data_agg = spotify_tracks.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in spotify_tracks.columns])
data_agg.show()

+----------+--------------+----------------+----------+----------------------------+-----------+------------------+------------------+-----------------+------------+----------------------+-----------+--------------+--------------+------------+-----------------+-----------+--------------------+-------------+-----------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---+-----+----+
|track_name|track_explicit|track_popularity|album_name|album_release_date_precision|artist_name|audio_acousticness|audio_danceability|audio_duration_ms|audio_energy|audio_instrumentalness|audio_key_1|audio_liveness|audio_loudness|audio_mode_1|audio_speechiness|audio_tempo|audio_time_signature|audio_valence|track_genre|pitch_1|pitch_2|pitch_3|pitch_4|pitch_5|pitch_6|pitch_7|pitch_8|pitch_9|pitch_10|pitch_11|pitch_12|timbre_1|timbre_2|timbre_3|timbre_4

In [27]:
del data_agg
spotify_tracks = spotify_tracks.dropna()

In [28]:
# Reverse json 

import json

# Opening JSON file
f = open('map_genre.json')
map_genre = json.load(f)

def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = key 
            #else: 
            #    inverse[item].append(key) 
    return inverse

map_genre_inv = invert_dict(map_genre)

In [29]:
udf_foo = udf(lambda x: map_genre_inv[x], StringType())
spotify_tracks = spotify_tracks.withColumn("track_genre_reduced", udf_foo("track_genre"))

In [30]:
TARGET_VARIABLE = "track_genre_reduced"

spotify_tracks.groupBy('track_genre_reduced').count().toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
track_genre_reduced,pop,alternative,ambient,electronic,hip-hop,nation,jazz,country,musical,metal,rock,classical,latin,unclassified
count,5268,5994,3129,13649,2311,6118,3632,4555,2547,3598,5258,3148,5028,645


In [31]:
spotify_tracks = spotify_tracks.filter(spotify_tracks.track_genre_reduced != 'unclassified')
spotify_tracks.groupBy('track_genre_reduced').count().toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12
track_genre_reduced,pop,alternative,ambient,electronic,hip-hop,nation,jazz,country,musical,metal,rock,classical,latin
count,5268,5994,3129,13649,2311,6118,3632,4555,2547,3598,5258,3148,5028


In [39]:
fractions_dict = spotify_tracks.groupBy('track_genre_reduced').count().toPandas().to_dict()

fraction = {}
for genre in fractions_dict["track_genre_reduced"]:
  g = fractions_dict["track_genre_reduced"][genre]
  # fraction[g] = fractions_dict["count"][genre] / spotify_tracks.count()
  fraction[g] = 1 / 13
 
fraction

{'alternative': 0.07692307692307693,
 'ambient': 0.07692307692307693,
 'classical': 0.07692307692307693,
 'country': 0.07692307692307693,
 'electronic': 0.07692307692307693,
 'hip-hop': 0.07692307692307693,
 'jazz': 0.07692307692307693,
 'latin': 0.07692307692307693,
 'metal': 0.07692307692307693,
 'musical': 0.07692307692307693,
 'nation': 0.07692307692307693,
 'pop': 0.07692307692307693,
 'rock': 0.07692307692307693}

In [40]:
#spotify_tracks = spotify_tracks.sampleBy("track_genre_reduced", fractions=fraction, seed=0)
#spotify_tracks.groupBy('track_genre_reduced').count().toPandas().transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12
track_genre_reduced,electronic,country,classical,musical,nation,pop,alternative,ambient,metal,hip-hop,rock,latin,jazz
count,234,37,7,9,50,32,45,13,19,12,36,30,19


In [34]:
# value counts of Batsman_Name column
spotify_tracks.groupBy('album_release_date_precision').count().show()

spotify_tracks.groupBy('track_name').count().show()

spotify_tracks.groupBy('album_name').count().show()

spotify_tracks.groupBy('artist_name').count().show()

+----------------------------+-----+
|album_release_date_precision|count|
+----------------------------+-----+
|                         day| 5796|
|                        year|  868|
+----------------------------+-----+

+--------------------+-----+
|          track_name|count|
+--------------------+-----+
|          Trap Queen|    1|
|        Aztec Mystic|    1|
|       You Were Mine|    1|
|         Separuh Aku|    1|
|Just Can't Stay A...|    1|
|           Dark Star|    1|
|           Treblinka|    1|
| I Know You Got Soul|    1|
|  'Running on Empty'|    1|
|  ABSOLUTE EGO DANCE|    1|
|             Company|    1|
|             Boiling|    1|
|       Le Tue Parole|    1|
|             Darlene|    1|
|Goodbye England (...|    1|
| Turn Off Your Phone|    1|
|           Tu Diseño|    1|
|Rally Round Jah T...|    1|
|     Good Vibrations|    1|
|   The Lord's Prayer|    1|
+--------------------+-----+
only showing top 20 rows

+----------------------------------+-----+
|           

In [35]:
# This function is responsible to implement the pipeline above for transforming categorical features into numerical ones
def to_numerical(df, numerical_features, categorical_features, target_variable):
  
    # 1. Label Encode target feature 
    label_indexer= StringIndexer(inputCol=target_variable, outputCol='label')

    # 2. Label Encode Categorical features
    indexer = [StringIndexer(inputCol=c, outputCol="{0}_index".format(c), handleInvalid="skip") for c in categorical_features]

    # 3. OneHot Encode 
    # stage_3 = OneHotEncoder(inputCol='album_release_date_precision_index', outputCol='album_release_date_precision_oh')

    # 4. create a vector of all the features required to train the logistic regression model 
    # encoded_columns = ['track_name_index', 'album_name_index', 'artist_name_index', 'album_release_date_precision_oh']
    # stage_4 = VectorAssembler(inputCols= encoded_columns + numerical_features, outputCol='features')
    assembler = VectorAssembler(inputCols= [indexer.getOutputCol() for indexer in indexer] + numerical_features, outputCol='features')
    
    # 4.a Create the StandardScaler
    scaler = StandardScaler(inputCol=assembler.getOutputCol(), outputCol="std_" + assembler.getOutputCol(), withStd=True, withMean=True)

   

    # 5. Populate the stages of the pipeline
    # stages = [stage_1] + stage_2 +[stage_3] + [stage_4] + [scaler]
    stages = [label_indexer] + indexer + [assembler] + [scaler]
    # 6. Setup the pipeline with the stages above
    pipeline = Pipeline(stages=stages)

    # 7. Transform the input dataframe accordingly
    transformer = pipeline.fit(df)
    df_transformed = transformer.transform(df)

    return df_transformed

In [36]:
# Transform the training set and get back both the transformer and the new dataset
spotify_tracks_encoded = to_numerical(spotify_tracks, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)
spotify_tracks_encoded.cache()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
# Clean-up unused variables
# Let's try to free-up some RAM
import gc
del spotify_tracks
# ...

print("Garbage collector: collected %d objects" % (gc.collect()))

In [ ]:
# Select `features` and `label` (i.e., formerly `deposit`) target variable only
spotify_tracks_final = spotify_tracks_encoded.select(["std_features", "label"])
spotify_tracks_final.cache()

del spotify_tracks_encoded

# BALANCEMENT for splitting
#sampled = spotify_tracks_final.sampleBy("label", fractions={4: 0.2, 6: 0.4, 8: 0.2}, seed=0)
#sampled.show()

RANDOM_SEED = 42
# Randomly split our original dataset `house_df` into 80÷20 for training and test, respectively
train_set, test_set = spotify_tracks_final.randomSplit([0.8, 0.2], seed=RANDOM_SEED)

train_set.show(5, truncate=False)
train_set.printSchema()

In [ ]:
del spotify_tracks_final
# ...

print("Garbage collector: collected %d objects" % (gc.collect()))

In [ ]:
# This function defines the general pipeline for logistic regression
def logistic_regression_pipeline(train):

    stage_5_lg = LogisticRegression(featuresCol='std_features',labelCol='label')

    logistic_regression_pipeline = Pipeline(stages= [stage_5_lg])

    #### LOGISTIC REGRESSION
    param_grid = ParamGridBuilder()\
    .addGrid(stage_5_lg.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(stage_5_lg.maxIter, [10, 20, 50]) \
    .build()
    # other param: .addGrid(stage_4_lg.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0])
    cross_val_lg = CrossValidator(estimator=logistic_regression_pipeline,
                            estimatorParamMaps=param_grid,
                            evaluator=MulticlassClassificationEvaluator().setMetricName("accuracy"), # default = "areaUnderROC", alternatively "areaUnderPR"
                            numFolds=5,
                            collectSubModels=True
                            )
    cv_model_lg = cross_val_lg.fit(train)

    return cv_model_lg

In [ ]:
# This function defines the general pipeline for logistic regression
def decision_tree_pipeline(train):
  
  stage_5_dr = DecisionTreeClassifier(featuresCol='std_features',labelCol='label')

  decision_tree_pipeline = Pipeline(stages= [stage_5_dr])

  #### DECISION TREE
  param_grid = ParamGridBuilder()\
    .addGrid(stage_5_dr.maxDepth, [3, 5, 8]) \
    .addGrid(stage_5_dr.impurity, ["gini", "entropy"]) \
    .build()
  cross_val_dt = CrossValidator(estimator=decision_tree_pipeline,
                                estimatorParamMaps=param_grid,
                                evaluator=MulticlassClassificationEvaluator().setMetricName("accuracy"), # default = "areaUnderROC", alternatively "areaUnderPR"
                                numFolds=5,
                                collectSubModels=True
                                )
  cv_model_dt = cross_val_dt.fit(train)

  return cv_model_dt

In [ ]:
# This function defines the general pipeline for logistic regression
def random_forest_pipeline(train):
  
  stage_5_rf = RandomForestClassifier(featuresCol="std_features", labelCol="label")

  random_forest_pipeline = Pipeline(stages= [stage_5_rf])

  #### RANDOM FOREST
  param_grid = ParamGridBuilder()\
    .addGrid(stage_5_rf.maxDepth, [3, 5, 8]) \
    .addGrid(stage_5_rf.numTrees, [10, 50, 100]) \
    .build()
  cross_val_rf = CrossValidator(estimator=random_forest_pipeline, 
                              estimatorParamMaps=param_grid,
                              evaluator= MulticlassClassificationEvaluator().setMetricName("accuracy"), # default = "areaUnderROC", alternatively "areaUnderPR"
                              numFolds=5,
                              collectSubModels=True 
                              )
  cv_model_rf = cross_val_rf.fit(train)

  return cv_model_rf

In [ ]:
print("Garbage collector: collected %d objects" % (gc.collect()))
cv_model_dt = decision_tree_pipeline(train_set)

# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions_dt = cv_model_dt.transform(test_set)

test_predictions_dt.select("std_features", "prediction", "label").show(5)

In [ ]:
for i, avg_roc_auc in enumerate(cv_model_dt.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))


print("Best model according to k-fold cross validation: maxDept=[{:d}]; impurity=[{:s}]".
      format(cv_model_dt.bestModel.stages[-1]._java_obj.getMaxDepth(), 
             cv_model_dt.bestModel.stages[-1]._java_obj.getImpurity(),
             )
      )
print(cv_model_dt.bestModel.stages[-1])

print("***** Test Set *****")
evaluator = cv_model_dt.getEvaluator()
print('Test F1-Score ',   evaluator.evaluate(test_predictions_dt, {evaluator.metricName: 'f1'}))
print('Test Precision ',  evaluator.evaluate(test_predictions_dt, {evaluator.metricName: 'weightedPrecision'}))
print('Test Recall ',     evaluator.evaluate(test_predictions_dt, {evaluator.metricName: 'weightedRecall'}))
print('Test Accuracy ',   evaluator.evaluate(test_predictions_dt, {evaluator.metricName: 'accuracy'}))

In [ ]:
#del cv_model_dt
print("Garbage collector: collected %d objects" % (gc.collect()))
cv_model_lg = logistic_regression_pipeline(train_set)

# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions_lg = cv_model_lg.transform(test_set)

test_predictions_lg.select("std_features", "prediction", "label").show(5)

In [ ]:
for i, avg_roc_auc in enumerate(cv_model_lg.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))


print("Best model according to k-fold cross validation: lambda=[{:.3f}]; alfa=[{:.3f}]".
      format(cv_model_lg.bestModel.stages[-1]._java_obj.getRegParam(), 
             cv_model_lg.bestModel.stages[-1]._java_obj.getElasticNetParam(),
             )
      )
print(cv_model_lg.bestModel.stages[-1])

print("***** Test Set *****")
evaluator = cv_model_lg.getEvaluator()
print('Test F1-Score ',   evaluator.evaluate(test_predictions_lg, {evaluator.metricName: 'f1'}))
print('Test Precision ',  evaluator.evaluate(test_predictions_lg, {evaluator.metricName: 'weightedPrecision'}))
print('Test Recall ',     evaluator.evaluate(test_predictions_lg, {evaluator.metricName: 'weightedRecall'}))
print('Test Accuracy ',   evaluator.evaluate(test_predictions_lg, {evaluator.metricName: 'accuracy'}))

In [ ]:
for i, avg_roc_auc in enumerate(cv_model_rf.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))


print("Best model according to k-fold cross validation: maxDept=[{:d}]".
      format(cv_model_rf.bestModel.stages[-1]._java_obj.getMaxDepth(), 
             )
      )
print(cv_model_rf.bestModel.stages[-1])

print("***** Test Set *****")
evaluator = cv_model_rf.getEvaluator()
print('Test F1-Score ',   evaluator.evaluate(test_predictions_rf, {evaluator.metricName: 'f1'}))
print('Test Precision ',  evaluator.evaluate(test_predictions_rf, {evaluator.metricName: 'weightedPrecision'}))
print('Test Recall ',     evaluator.evaluate(test_predictions_rf, {evaluator.metricName: 'weightedRecall'}))
print('Test Accuracy ',   evaluator.evaluate(test_predictions_rf, {evaluator.metricName: 'accuracy'}))